# Image_Feature_Maps_TensorFlow

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [資料處理參數設定](#3)
1. [資料處理](#4)
    -  [載入模型](#4.1)
    -  [生成完整模型特徵圖](#4.2)
    -  [生成模型指定層特徵圖](#4.3)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
# !pip install -q efficientnet
# import efficientnet.tfkeras as efn

In [ ]:
# !pip install tensorflow-addons
# import tensorflow_addons.optimizers as addons_optimizers

In [ ]:
# # 資料處理套件
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus']=False # 用來正常顯示負號
plt.rcParams.update({'figure.max_open_warning': 0}) # 關閉開啟figure數量20以上的警告

In [ ]:
# # # tensorflow深度學習模組套件
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.models import Model
from   tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.python.client import device_lib

In [ ]:
np.seterr(divide='ignore', invalid='ignore') # 在NumPy中忽略除以0的警告

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
# 查看設備
print(device_lib.list_local_devices())

In [ ]:
# 查看tensorflow版本
print(tf.__version__)

# 查看圖像通道位置
print(K.image_data_format())

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = True

# (Boolean)是否為 Colab
COLAB = False

# (String)CPU/GPU/TPU
DEVICE = "GPU"


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    
# (String)資料根路徑
# DATA_ROOT_PATH = PATH+r'datasets/AI_CUP_2020_AIMango_Grade_Classification/' 
DATA_ROOT_PATH = PATH+r'datasets/faces_glintasia_112x112_folders/' 

# (String)測試單張圖片檔案路徑
# TRAIN_FILE_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Test_Cropped'
TRAIN_FILE_DATA_PATH = DATA_ROOT_PATH+r'2'
    
# (String)測試單張圖片檔名
# IMAGE_PATH =  TRAIN_FILE_DATA_PATH + '/00001.jpg'
IMAGE_PATH =  TRAIN_FILE_DATA_PATH + '/34.jpg'

# # (String)儲存層特徵圖的路徑檔名
LAYER_FEATURE_MAP_PATH = "layer_feature_map.png"

# # (String)儲存完整模型特徵圖的路徑檔名
MODEL_FEATURE_MAP_PATH = "layer_model_feature_map.png"

# (String)模型名稱(使用哪個模型)
# MODEL_NAME = 'efficientnetb7'
# MODEL_NAME = 'se_mobile_facenet_glintasia_1000'
MODEL_NAME = 'se_mobile_facenet_glintasia_2000'

# (String)專案名稱
# PROJECT_NAME = 'AI_CUP_2020_AIMango_Grade_Classification'
PROJECT_NAME = 'Face_Recognition'

# (String)模型的專案路徑
PROJECT_PATH = PATH + r'models/'+PROJECT_NAME

# (String)讀取模型的儲存路徑 
LOAD_MODEL_PATH = PROJECT_PATH + r'/' + MODEL_NAME + '.h5'

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if DEVICE != "CPU":
    !nvidia-smi

In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
# 動態申請顯存
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

# 3. 資料處理參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)圖片尺寸
# IMAGE_SIZE = 224
IMAGE_SIZE = 112

# (Int)生層哪一層特徵圖，第一層從零開始
LAYER = 1
    
# (Boolean)是否印出完整模型(完整模型特徵圖模型)
MODEL_PRINT = False

# (Boolean)是否印出層特徵圖模型
LAYER_FEATURE_MAP_MODEL_PRINT = True

# (Boolean)是否生成完整模型特徵圖
GENERATE_MODEL_FEATURE_MAP = False

# (Boolean)是否生成層特徵圖
GENERATE_LAYER_FEATURE_MAP = True

# (Boolean)是否儲存完整模型特徵圖
SAVE_MODEL_FEATURE_MAP = False

# (Boolean)是否儲存層特徵圖
SAVE_LAYER_FEATURE_MAP = False


''''圖表參數設定'''

# (Folat)模型特徵圖顯示縮放係數
MODEL_FEATURE_MAP_SCALE = 20.

# (Int)層特徵圖列表寬度
LAYER_FEATURE_MAP_FIGSIZE_W = 50

# (Int)層特徵圖列表高度
LAYER_FEATURE_MAP_FIGSIZE_H = 50

# (Int)層特徵圖列表列數，請參考layer.shape[3]整除設定，(layer.shape[0], layer.shape[1], layer.shape[2], layer.shape[3])
LAYER_FEATURE_MAP_SUBPLOT_COULMN = 8

# (Int)層特徵圖列表行數，請參考layer.shape[3]整除設定，(layer.shape[0], layer.shape[1], layer.shape[2], layer.shape[3])
LAYER_FEATURE_MAP_SUBPLOT_ROW = 8

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入模型 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
print("Reading the pre-trained model... ")
model = keras.models.load_model(LOAD_MODEL_PATH)
print("Reading done. ")

if MODEL_PRINT:
    model.summary()

## 4.2 生成完整模型特徵圖 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
if GENERATE_MODEL_FEATURE_MAP:
    # Let's define a new Model that will take an image as input, and will output
    # intermediate representations for all layers in the previous model after
    # the first.
    successive_outputs = [layer.output for layer in model.layers[1:]]

    #visualization_model = Model(img_input, successive_outputs)
    visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

    img = load_img(IMAGE_PATH, target_size=(IMAGE_SIZE, IMAGE_SIZE))  # this is a PIL image

    x   = img_to_array(img) # Numpy array with shape (150, 150, 3)
    x   = x.reshape((1,) + x.shape) # Numpy array with shape (1, 150, 150, 3)

    # Rescale by 1/255
    x /= 255.0

    # Let's run our image through our network, thus obtaining all
    # intermediate representations for this image.
    successive_feature_maps = visualization_model.predict(x)

    # These are the names of the layers, so can have them as part of our plot
    layer_names = [layer.name for layer in model.layers]

    count = 1
    # Now let's display our representations
    for layer_name, feature_map in zip(layer_names, successive_feature_maps):
        print(feature_map.shape)
        if len(feature_map.shape) == 4:

            # Just do this for the conv / maxpool layers, not the fully-connected layers
            n_features = feature_map.shape[-1]  # number of features in the feature map
            size = feature_map.shape[1]  # feature map shape (1, size, size, n_features)

            # We will tile our images in this matrix
            display_grid = np.zeros((size, size * n_features))

            # Postprocess the feature to be visually palatable
            for i in range(n_features):
                x  = feature_map[0, :, :, i]
                x -= x.mean()
                x /= x.std ()
                x *=  64
                x += 128
                x  = np.clip(x, 0, 255).astype('uint8')
                display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

            # Display the grid
            scale = MODEL_FEATURE_MAP_SCALE / n_features
            plt.figure(figsize=(scale * n_features, scale))
            plt.title (layer_name)
            plt.grid  (False)
            plt.imshow(display_grid, aspect='auto', cmap='viridis')
            
            if SAVE_MODEL_FEATURE_MAP:
                plt.savefig(str(count)+MODEL_FEATURE_MAP_PATH)
                plt.show()
                count+=1

## 4.3 生成模型指定層特徵圖 <a class="anchor" id="4.3"></a>
[Back to Table of Contents](#0)

In [ ]:
if GENERATE_LAYER_FEATURE_MAP:
    inp= model.inputs 
    print(inp)
    out1= model.layers[LAYER]. output  
    print(out1)
    feature_map_1= Model(inputs= inp, outputs= out1)  
    
    if LAYER_FEATURE_MAP_MODEL_PRINT:
        feature_map_1.summary()

In [ ]:
if GENERATE_LAYER_FEATURE_MAP:
    img = cv2.imread(IMAGE_PATH) # Reading an Image
    img = cv2.resize(img,(IMAGE_SIZE,IMAGE_SIZE)) # Resizing an Image
    input_img = np.expand_dims(img, axis=0) # Expanding the dimension
    print(input_img.shape) # Printing out the size of the Input Image

    f1 = feature_map_1.predict(input_img) # predicting out the Image 
    print(f1.shape) # Let's see the shape

    fig = plt.figure(figsize = (LAYER_FEATURE_MAP_FIGSIZE_W,LAYER_FEATURE_MAP_FIGSIZE_H))
    for i in range(f1.shape[3]):
        ax = fig.add_subplot(LAYER_FEATURE_MAP_SUBPLOT_COULMN,LAYER_FEATURE_MAP_SUBPLOT_ROW,i + 1)
        ax.imshow(f1[0,:,:,i])

    if SAVE_LAYER_FEATURE_MAP:
        plt.savefig(LAYER_FEATURE_MAP_PATH)
        plt.show()